# 4 - Modelagem com Machine Learning

P1: É possível predizer a taxa de poluição emitida no ar nos próximos anos?\
P2: A estação do ano influencia a quantidade de poluentes no ar de alguma forma?

1 - tentar fazer previsões utilziando modelos lineares para ambos perguntas 1 e 2

2 - tentar verificar se eles conseguem prever com corretude anos posteriores ao treinamento do modelo.

3 - Aplicar modelos mais complexos para as mesmas coisas que a regressão linear tentou verificar

4 - verificar se as regressões mais complexas conseguem prever melhor que a regressão linear 

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression, r_regression, RFECV
from sklearn.impute import SimpleImputer
from sklearn.model_selection import TimeSeriesSplit 

import duckdb
import pandas as pd
import numpy as np
import os 

PM25_ID = 365
NO2_ID = 375
O3_ID = 386

SPLIT_YEAR=2020

BOROS = ["Bronx", "Brooklyn", "Manhattan", "Queens", "Staten Island"]

OUT_FOLDER = "./out"

In [17]:
def calc_KBest(features: list, target_col: str, df: pd.DataFrame):
    X = df[features]
    y = df[target_col]

    imputer = SimpleImputer(strategy='mean')
    X_imp = imputer.fit_transform(X)

    selector = SelectKBest(f_regression, k=3).fit(X_imp, y)

    feat_scores = pd.Series(selector.scores_, index=features)
    feat_scores.sort_values(ascending=False, inplace=True)

    return feat_scores

def calc_RFECV(features: list, target_col: str, df: pd.DataFrame, features_qtd: int = 3, model  = LinearRegression()):
    df = df.dropna().copy()

    n_samples = min(len(df) - 1, 3)

    X = df[features]
    y = df[target_col]

    imputer = SimpleImputer(strategy='mean')
    X_imp = imputer.fit_transform(X)

    selector = RFECV(estimator=model, cv=TimeSeriesSplit(n_samples), scoring='r2', min_features_to_select=features_qtd)
    selector.fit(X_imp, y)

    feat_scores = pd.Series(selector.ranking_, index=features)
    feat_scores.sort_values(ascending=False, inplace=True)

    return feat_scores

def linear_regression(data: pd.DataFrame, features: list, target_col: str, split_year: int, pollutant_id: int):
    df_filtered = data[data['IndicatorID'] == pollutant_id].dropna().copy()

    X_train = df_filtered[df_filtered['Year'] < split_year][features]
    y_train = df_filtered[df_filtered['Year'] < split_year][target_col]
    X_test = df_filtered[df_filtered['Year'] >= split_year][features]
    y_test = df_filtered[df_filtered['Year'] >= split_year][target_col]
    
    # Padronização (Scaling)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Treinamento do Modelo
    model = LinearRegression()
    model.fit(X_train_scaled, y_train)
    
    # Predição e Avaliação
    y_pred = model.predict(X_test_scaled)
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    # Coeficientes (Betas Padronizados)
    betas = pd.Series(model.coef_, index=features).sort_values(ascending=False)
    
    # Output
    pollutant_name = df_filtered['Name'].iloc[0] # Pega o nome do poluente
    print("-------------------------------------------------")
    print(f"## Resultados RL | {pollutant_name} ({df_filtered['GeoPlaceName'].iloc[0]})")
    print(f"Período Treino: {df_filtered['Year'].min()} - {split_year-1} | Teste: {split_year} - {df_filtered['Year'].max()}")
    print(f"R-quadrado (R²): {r2:.3f}")
    print(f"RMSE: {rmse:.3f}")
    print("\n## Coeficientes:")
    print(betas)
    print("-------------------------------------------------")

def rf_regression(data: pd.DataFrame, features: list, target_col: str, split_year: int, pollutant_id: int):
    df_filtered = data[data['IndicatorID'] == pollutant_id].copy()

    # Separação Treino/Teste por ano
    X_train = df_filtered[df_filtered['Year'] < split_year][features]
    y_train = df_filtered[df_filtered['Year'] < split_year][target_col]
    X_test = df_filtered[df_filtered['Year'] >= split_year][features]
    y_test = df_filtered[df_filtered['Year'] >= split_year][target_col]
    
    # Treinamento do Modelo
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Predição e Avaliação
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    # Importância das Features
    importances = pd.Series(model.feature_importances_, index=features).sort_values(ascending=False)
    
    # Output
    pollutant_name = df_filtered['Name'].iloc[0] # Pega o nome do poluente
    print("-------------------------------------------------")
    print(f"## Resultados RF | {pollutant_name} (({df_filtered['GeoPlaceName'].iloc[0]}))")
    print(f"Período Treino: {df_filtered['Year'].min()} - {split_year-1} | Teste: {split_year} - {df_filtered['Year'].max()}")
    print(f"R-quadrado (R²): {r2:.3f}")
    print(f"RMSE: {rmse:.3f}")
    print("\n## Importância das Features:")
    print(importances)
    print("-------------------------------------------------")

def gradient_boosting_regression(data: pd.DataFrame, features: list, target_col: str, split_year: int, pollutant_id: int):
    df_filtered = data[data['IndicatorID'] == pollutant_id].dropna().copy()

    # Separação Treino/Teste por ano
    X_train = df_filtered[df_filtered['Year'] < split_year][features]
    y_train = df_filtered[df_filtered['Year'] < split_year][target_col]
    X_test = df_filtered[df_filtered['Year'] >= split_year][features]
    y_test = df_filtered[df_filtered['Year'] >= split_year][target_col]
    
    # Treinamento do Modelo
    model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
    model.fit(X_train, y_train)
    
    # Predição e Avaliação
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    # Importância das Features
    importances = pd.Series(model.feature_importances_, index=features).sort_values(ascending=False)
    
    # Output
    pollutant_name = df_filtered['Name'].iloc[0] # Pega o nome do poluente
    print("-------------------------------------------------")
    print(f"## Resultados GB | {pollutant_name} (({df_filtered['GeoPlaceName'].iloc[0]}))")
    print(f"Período Treino: {df_filtered['Year'].min()} - {split_year-1} | Teste: {split_year} - {df_filtered['Year'].max()}")
    print(f"R-quadrado (R²): {r2:.3f}")
    print(f"RMSE: {rmse:.3f}")
    print("\n## Importância das Features:")
    print(importances)
    print("-------------------------------------------------")

# Feature selection de dados Borough


In [18]:
P1_seasonal_borough = pd.read_parquet(os.path.join(OUT_FOLDER, "seasonal_borough_air_quality.parquet"))

sql_query = f"""
with base as(
    select 
        indicatorid,
        name,
        year,
		geoplacename,
        AVG(datavalue) AS DataValue
    from P1_seasonal_borough
    group by indicatorid, name, year, geoplacename
)
select
    indicatorid,
    name,
    year,
		geoplacename,
    DataValue as Current,
    lag(DataValue, 1) over (
            partition by indicatorid, geoplacename
            order by year
        ) as Lag1,
    lag(DataValue, 2) over (
            partition by indicatorid, geoplacename
            order by year
        ) as Lag2,
    lag(DataValue, 3) over (
            partition by indicatorid, geoplacename
            order by year
        ) as Lag3
from base
order by year, name;
"""

df_query = duckdb.query(sql_query).to_df()

TARGET_COL = 'Current'
FEATURES = ['Lag1','Lag2','Lag3','Year']

df_PM25 = df_query[df_query['IndicatorID'] == PM25_ID]
df_NO2 = df_query[df_query['IndicatorID'] == NO2_ID]
df_O3 = df_query[df_query['IndicatorID'] == O3_ID]


In [19]:
for df in [df_PM25, df_NO2, df_O3]:
    for boro in BOROS:
        df_filtered_boro = df_query[df_query['GeoPlaceName'] == boro] 
        train_df = df_filtered_boro[df_filtered_boro['Year'] < SPLIT_YEAR]
        
        res = calc_RFECV(
            features=FEATURES,
            target_col=TARGET_COL,
            df=train_df,
            features_qtd=1,
            model=LinearRegression()
        )

        linear_regression(
            data=df_filtered_boro, 
            features=res.index.tolist()[:1],
            target_col=TARGET_COL,
            split_year=SPLIT_YEAR,
            pollutant_id=df_filtered_boro['IndicatorID'].iloc[0] 
        )

-------------------------------------------------
## Resultados RL | Fine particles (PM 2.5) (Bronx)
Período Treino: 2012 - 2019 | Teste: 2020 - 2023
R-quadrado (R²): -3.095
RMSE: 0.668

## Coeficientes:
Lag1    0.761907
dtype: float64
-------------------------------------------------
-------------------------------------------------
## Resultados RL | Fine particles (PM 2.5) (Brooklyn)
Período Treino: 2012 - 2019 | Teste: 2020 - 2023
R-quadrado (R²): -2.994
RMSE: 0.824

## Coeficientes:
Lag1    0.644306
dtype: float64
-------------------------------------------------
-------------------------------------------------
## Resultados RL | Fine particles (PM 2.5) (Manhattan)
Período Treino: 2012 - 2019 | Teste: 2020 - 2023
R-quadrado (R²): -1.588
RMSE: 0.508

## Coeficientes:
Year   -1.024397
dtype: float64
-------------------------------------------------
-------------------------------------------------
## Resultados RL | Fine particles (PM 2.5) (Queens)
Período Treino: 2012 - 2019 | Tes

In [20]:
for df in [df_PM25, df_NO2, df_O3]:
    for boro in BOROS:
        df_boro = df[df['GeoPlaceName'] == boro]

        for model in [LinearRegression(), RandomForestRegressor(), GradientBoostingRegressor()]:
            train_df = df_boro[df_boro['Year'] < SPLIT_YEAR] #evitar data leakage

            res = calc_RFECV(
                features=FEATURES,
                target_col=TARGET_COL,
                df=train_df,
                model=model
            )

            model_name = model.__class__.__name__

            if model_name == "LinearRegression":
                linear_regression(
                    data=df_boro,
                    features=res.index.tolist()[:3],
                    target_col=TARGET_COL,
                    split_year=SPLIT_YEAR,
                    pollutant_id=df_boro['IndicatorID'].iloc[0]
                )

            if model_name == "RandomForestRegressor":
                rf_regression(
                    data=df_boro,
                    features=res.index.tolist()[:3],
                    target_col=TARGET_COL,
                    split_year=SPLIT_YEAR,
                    pollutant_id=df_boro['IndicatorID'].iloc[0]
                )

            if model_name == "GradientBoostingRegressor":
                gradient_boosting_regression(
                    data=df_boro,
                    features=res.index.tolist()[:3],
                    target_col=TARGET_COL,
                    split_year=SPLIT_YEAR,
                    pollutant_id=df_boro['IndicatorID'].iloc[0]
            )
                

-------------------------------------------------
## Resultados RL | Fine particles (PM 2.5) (Bronx)
Período Treino: 2012 - 2019 | Teste: 2020 - 2023
R-quadrado (R²): -4.126
RMSE: 0.748

## Coeficientes:
Lag3    0.578526
Lag2    0.432390
Lag1    0.071685
dtype: float64
-------------------------------------------------
-------------------------------------------------
## Resultados RF | Fine particles (PM 2.5) ((Bronx))
Período Treino: 2009 - 2019 | Teste: 2020 - 2023
R-quadrado (R²): -6.227
RMSE: 0.888

## Importância das Features:
Lag1    0.473642
Lag2    0.301787
Lag3    0.224571
dtype: float64
-------------------------------------------------
-------------------------------------------------
## Resultados GB | Fine particles (PM 2.5) ((Bronx))
Período Treino: 2012 - 2019 | Teste: 2020 - 2023
R-quadrado (R²): -1.020
RMSE: 0.469

## Importância das Features:
Lag1    0.920625
Lag3    0.075160
Lag2    0.004214
dtype: float64
-------------------------------------------------
------------